In [1]:
with open("inputs/Day_19.txt") as f:
    raw_input_data = f.read()

In [6]:
from collections import namedtuple, defaultdict
from enum import Enum

    
def part_1_solution(raw_input):
    puzzle_input = list(map(int, raw_input.split(',')))
    
    affected_cnt = run_simulation(puzzle_input)
    
    return affected_cnt


def run_simulation(program):
    blocks_cnt = 0

    affected_cnt = 0
    for x in range(50):
        for y in range(50):
            if get_status(x, y, program):
                affected_cnt += 1
            
    return affected_cnt

def get_status(x, y, program):
    computer = start_computer(program)
    next(computer)
    computer.send(x)
    raw_output = computer.send(y)

    return raw_output.status
    
Finished = object()
Input = object()
Output = namedtuple('Output', ('status'))


def start_computer(sequence):
    index = 0
    relative_base = 0
    memory = defaultdict(int)
    
    for i, value in enumerate(sequence):
        memory[i] = value
        
    diag_nbr = None
    while True:
        opt_code = memory[index]
        opt_code_with_modes = str(opt_code).zfill(5)
        opt_code = int(opt_code_with_modes[-2:])
        modes = opt_code_with_modes[:-2]
        
        
        par_1_address = get_parameter_address(memory, modes[-1], index + 1, relative_base)
        par_2_address = get_parameter_address(memory, modes[-2], index + 2, relative_base)
        par_3_address = get_parameter_address(memory, modes[-3], index + 3, relative_base)
        
        par_1 = memory[par_1_address]
        par_2 = memory[par_2_address]
        
        if opt_code == 99:
            break
        elif opt_code == 3:
            target_address = par_1_address
                
            memory[target_address] = yield Input
            index += 2
        elif opt_code == 4:
            diag_nbr = par_1
            
            yield Output(diag_nbr)
                
            index += 2
        elif opt_code == 1:
            target_address = par_3_address
            memory[target_address] = par_1 + par_2
            index += 4
        elif opt_code == 2:
            target_address = par_3_address
            memory[target_address] = par_1 * par_2
            index += 4
        elif opt_code == 5:
            if par_1 != 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 6:
            if par_1 == 0:
                index = par_2
            else:
                index += 3
        elif opt_code == 7:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 < par_2 else 0
            index += 4
        elif opt_code == 8:
            target_address = par_3_address
            memory[target_address] = 1 if par_1 == par_2 else 0
            index += 4
        elif opt_code == 9:
            relative_base += par_1
            index += 2
        else:
            print(f"Wrong code: {opt_code}")  
    
    yield Finished

            
def get_parameter_address(memory, mode, par_index, relative_base):
    if mode == "0":
        return memory[par_index]
    elif mode == "1":
        return par_index
    elif mode == "2":
        return relative_base + memory[par_index]
    else:
        print(f"[ERROR] Wrong mode code: {mode}")

In [7]:
print(f"Part 1 solution: {part_1_solution(raw_input_data)}")

Part 1 solution: 176
 

In [17]:
with open("inputs/Day_19.txt") as fp: origprog = { i: int(x) for i, x in enumerate(fp.readline().split(',')) }

def check_point(x,y):
    global origprog
    prog = dict(origprog)
    pc=0
    rb=0
    inp = [x,y]
    while True:
        (modes, cmd), args = divmod(prog[pc], 100), []
        for (i,iswrite) in enumerate([[],[0,0,1],[0,0,1],[1],[0],[0,0],[0,0],[0,0,1],[0,0,1],[0]][cmd%99]):
            args += [prog[pc+i+1]]
            if modes % 10 == 2: args[-1] += rb
            if modes % 10 != 1 and iswrite == 0: args[-1] = prog.get(args[-1],0)
            modes //= 10
        orig_pc = pc
        if cmd == 1: prog[args[2]] = args[0] + args[1] # add
        elif cmd == 2: prog[args[2]] = args[0] * args[1] # multiply
        elif cmd == 3: prog[args[0]] = inp.pop(0) # input
        elif cmd == 4: #output
            return args[0]
        elif cmd == 5 and args[0] != 0: pc = args[1] # branch if true
        elif cmd == 6 and args[0] == 0: pc = args[1] # branch if false
        elif cmd == 7: prog[args[2]] = int(args[0] < args[1]) # test less than
        elif cmd == 8: prog[args[2]] = int(args[0] == args[1]) # test equal
        elif cmd == 9: rb += args[0] # adjust relative base
        elif cmd == 99: break # halt
        if pc == orig_pc: pc += len(args) + 1 #only go to the next instruction if we didn't jump

y=99
x=0
while True:
    while check_point(x,y) == 0:
        x += 1
    x2 = x + 99
    y2 = y -99
    if check_point(x2,y2) == 1:
        print('Part 2 solution: %d' % (x*10000+y2))
        break
    y += 1

Part 2 solution: 6751081
